# LangGraph Tweet Revision Loop

This notebook demonstrates a structured, iterative loop between tweet **generation** and **critique** using [LangGraph](https://github.com/langchain-ai/langgraph) and an OpenAI chat model.

The flow simulates a conversation between:
- An **AI assistant** that generates or revises a tweet
- A **"simulated human" critic** who offers feedback on how to improve it

LangGraph orchestrates this back-and-forth using a stateful message-passing graph, looping until a stopping condition is met (e.g., maximum number of turns).

### Key Concepts
- `ChatPromptTemplate` defines roles for generation and critique
- `LangGraph` controls the flow between nodes (`generate → reflect → generate`)
- Each node logs its output to show how the tweet evolves over time

---

**Inspired by the LangGraph Udemy course**  
[https://www.udemy.com/course/langgraph](https://www.udemy.com/course/langgraph)

In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import List, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import END, MessageGraph

load_dotenv()

True

In [2]:
# Define the prompt template for the reflection (critique) phase.
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a viral twitter influencer grading a tweet. Generate critique and recommendations for the user's tweet."
            "Always provide detailed recommendations, including requests for length, virality, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# Define the prompt template for the generation phase.
generation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a twitter techie influencer assistant tasked with writing excellent twitter posts."
            " Generate the best twitter post possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# Instantiate the LLM and bind each prompt to the model.
# This creates two distinct chains: one for tweet generation and one for reflection.
llm = ChatOpenAI()
generate_chain = generation_prompt | llm
reflect_chain = reflection_prompt | llm

In [3]:
# Define string constants for node names
REFLECT = "reflect"
GENERATE = "generate"

# Define the generation node logic.
# This prints the current state, invokes the generation chain,
# and returns an AIMessage containing the new tweet.
def generation_node(state: Sequence[BaseMessage]):
    # Get last user input and latest critique
    print("\n=== GENERATION NODE ===")
    for msg in state:
        print(f"{msg.type.upper()}: {msg.content}")

    result = generate_chain.invoke({"messages": state})
    print(f"\nGENERATION OUTPUT: {result.content}")

    return AIMessage(content=result.content)

# Define the reflection node logic.
# It trims the conversation to just the initial prompt and latest AI output,
# then uses the reflection chain to simulate a critique,
# returning it as a new HumanMessage (as if a user gave feedback).
def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    print("\n=== REFLECTION NODE ===")
    for msg in messages:
        print(f"{msg.type.upper()}: {msg.content}")

    # Only keep the original prompt and the last AI message
    # [first human msg, last AI message]
    filtered_messages = [messages[0], next(m for m in reversed(messages) if isinstance(m, AIMessage))]

    result = reflect_chain.invoke({"messages": filtered_messages})
    print(f"\nREFLECTION OUTPUT: {result.content}")

    return [HumanMessage(content=result.content)]

# Build the LangGraph by registering nodes and defining flow.
builder = MessageGraph()
builder.add_node(GENERATE, generation_node)
builder.add_node(REFLECT, reflection_node)
builder.set_entry_point(GENERATE)

# Define the loop condition:
# If the message history is longer than 6, stop.
# Otherwise, route to the reflection node.
def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        return END
    return REFLECT

# Add conditional logic:
# generate → reflect (loop) or → END (stop)
builder.add_conditional_edges(GENERATE, should_continue)
builder.add_edge(REFLECT, GENERATE)

# Compile the graph for execution.
graph = builder.compile()

In [5]:
# Start execution
print("Running LangGraph")

# Define the initial input message (the user's tweet to improve).
# This is wrapped as a HumanMessage and passed as the starting state to the graph.
inputs = HumanMessage(content="""Make this tweet better:"
                                @LangChainAI
        — newly Tool Calling feature is seriously underrated.

        After a long wait, it's  here- making the implementation of agents across different models with function calling - super easy.

                              """)

# Invoke the compiled LangGraph with the input.
# This triggers the generate → reflect → generate... loop until the stopping condition is met.
response = graph.invoke(inputs)

Running LangGraph

=== GENERATION NODE ===
HUMAN: Make this tweet better:"
                                @LangChainAI
        — newly Tool Calling feature is seriously underrated.

        After a long wait, it's  here- making the implementation of agents across different models with function calling - super easy.

                              

GENERATION OUTPUT: "🔗 Don't overlook @LangChainAI's latest Tool Calling feature! This game-changer streamlines agent implementation across various models with simple function calling. ✨ #TechInnovation #AI"

=== REFLECTION NODE ===
HUMAN: Make this tweet better:"
                                @LangChainAI
        — newly Tool Calling feature is seriously underrated.

        After a long wait, it's  here- making the implementation of agents across different models with function calling - super easy.

                              
AI: "🔗 Don't overlook @LangChainAI's latest Tool Calling feature! This game-changer streamlines agent implemen

In [6]:
# Print the final output from the graph — typically the last revised tweet.
print(response[-1].content)

"🔗 Explore @LangChainAI's new Tool Calling feature, simplifying agent implementation across models effortlessly! Boost your workflow efficiency and performance with this innovation. ✨ #AI #TechInnovation #SoftwareDevelopment

Ready to revolutionize your approach? Share your thoughts below! 👇"
